<a href="https://colab.research.google.com/github/chanheum/machinerunning/blob/main/with_hugging_face_240829.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
!python --version

Python 3.10.12


In [37]:
!pip install mxnet==1.9.1
!pip install gluonnlp==0.10.0
!pip install pandas==1.5.3
!pip install tqdm==4.66.1
!pip install sentencepiece==0.1.99
# !pip install transformers==4.34.0
# !pip install transformers
!pip install transformers==4.37.0
!pip install huggingface-hub==0.23.0
!pip install torch==2.0.1
!pip install numpy==1.23.1

In [38]:
!pip install firebase-admin
!pip show firebase-admin

Name: firebase-admin
Version: 6.5.0
Summary: Firebase Admin Python SDK
Home-page: https://firebase.google.com/docs/admin/setup/
Author: Firebase
Author-email: 
License: Apache License 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: cachecontrol, google-api-core, google-api-python-client, google-cloud-firestore, google-cloud-storage, pyjwt
Required-by: firebase-functions


In [39]:
!pip install firebase_functions

python version : 3.10.12

<Modules Version>
mxnet : 1.9.1

gluonnlp : 0.10.0

pandas : 1.5.3

tqdm : 4.66.1

sentencepiece : 0.1.99

transformers : 4.34.0

torch : 2.0.1+cu118

numpy : 1.23.1

In [40]:
# ★
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-tmmtsef5/kobert-tokenizer_6e79a083d1664cc3a888d9a0a474ec58
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-tmmtsef5/kobert-tokenizer_6e79a083d1664cc3a888d9a0a474ec58
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done


In [41]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import time

In [42]:
# ★ Hugging Face를 통한 모델 및 토크나이저 Import
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [43]:
# GPU 사용 시
device = torch.device("cuda:0")

In [44]:
# ★
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [45]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [46]:
# 데이터 불러오기
# !wget https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
# !wget https://www.dropbox.com/s/977gbwh542gdy94/ratings_test.txt?dl=1

# 데이터 불러오기 (나)
import pandas as pd
# chatbot_data = pd.read_csv('/content/drive/MyDrive/sentences.csv', encoding='cp949')
# chatbot_data = pd.read_csv('/content/drive/MyDrive/sentences_custom_240708.csv', encoding='cp949')      # 파일명 수정
# chatbot_data = pd.read_csv('/content/drive/MyDrive/sentences_custom_240828.csv', encoding='cp949')      # 2024-08-28 추가
# chatbot_data = pd.read_csv('/content/drive/MyDrive/sentences_custom_240828.csv', encoding='UTF-8)')      # 2024-08-28 추가
chatbot_data = pd.read_csv('/content/drive/MyDrive/sentences_custom_240829.csv', encoding='UTF-8)')      # 2024-08-29 추가

In [47]:
chatbot_data.loc[(chatbot_data['의도'] == "neutral"), '의도'] = 0           #일상대화  => 0
chatbot_data.loc[(chatbot_data['의도'] == "call"), '의도'] = 1              #코웨이 AI 호출  => 1
chatbot_data.loc[(chatbot_data['의도'] == "power_on"), '의도'] = 2          #전원 on => 2
chatbot_data.loc[(chatbot_data['의도'] == "power_off"), '의도'] = 3         #전원 off  => 3
chatbot_data.loc[(chatbot_data['의도'] == "mode_goodmoring"), '의도'] = 4   #굿모닝모드 => 4
chatbot_data.loc[(chatbot_data['의도'] == "mode_goodday"), '의도'] = 5      #굿데이모드  => 5
chatbot_data.loc[(chatbot_data['의도'] == "mode_goodnight"), '의도'] = 6    #굿나잇모드 => 6
chatbot_data.loc[(chatbot_data['의도'] == "recline_zerog"), '의도'] = 7     #무중력모드 => 7
chatbot_data.loc[(chatbot_data['의도'] == "recline_swing"), '의도'] = 8     #스윙모드 => 8
chatbot_data.loc[(chatbot_data['의도'] == "heat_on"), '의도'] = 9           #온열 on => 9
chatbot_data.loc[(chatbot_data['의도'] == "heat_off"), '의도'] = 10         #온열 off  => 10

# 2024-08-29 추가
chatbot_data.loc[(chatbot_data['의도'] == "agree"), '의도'] = 11                    #사용자_동의  => 11
chatbot_data.loc[(chatbot_data['의도'] == "disagree"), '의도'] = 12                 #사용자_비동의  => 12
chatbot_data.loc[(chatbot_data['의도'] == "user_tired"), '의도'] = 13               #사용자_상태_피곤함  => 13
chatbot_data.loc[(chatbot_data['의도'] == "user_need_neck_msg"), '의도'] = 14       #사용자_목안마_필요  => 14
chatbot_data.loc[(chatbot_data['의도'] == "user_need_shoulder_msg"), '의도'] = 15   #사용자_어깨안마_필요  => 15
chatbot_data.loc[(chatbot_data['의도'] == "user_need_back_msg"), '의도'] = 16       #사용자_등안마_필요  => 16

In [48]:
chatbot_data.sample(n=10)

,발화문,의도
1447,온열오프.,10
3147,알았어. 영화를 주로 찾아서 볼게.,0
3135,실망스러운 건 본인들 스스로 사과를 안했다는 거야.,0
5732,헐. 나 이벤트에 당첨되었어.,0
5301,지금 팔 수도 없어. 손해가 너무 심하거든.,0
398,하이 코웨이.,1
5188,주변 사람들한테 이벤트에 당첨돼서 비싼 향수를 받았다고 자랑했어. 다들 부러워 해.,0
5378,나 축하해줘. 기분 좋은 일이 생겼어.,0
3699,그렇잖아도 내일 만나기로 약속했어.,0
2672,지금 노트북을 여기저기 살펴봤는데 전원도 잘 켜지고 작동도 잘 돼. 문제 있는 곳이...,0


In [49]:
data_list = []
for q, label in zip(chatbot_data['발화문'], chatbot_data['의도'])  :
    if(type(q) != float):
        data = []
        data.append(q)
        data.append(str(label))

        data_list.append(data)
    else:
        pass

In [50]:
print(len(data_list))
print(data_list[0])
# print(data_list[6000])
# print(data_list[12000])
print(data_list[-1])

8041
['하이 코웨이.', '1']
['아닝.', '12']


In [51]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, shuffle=True, random_state=34)

In [52]:
print(len(dataset_train))
print((dataset_train))

6030
[['진짜 덥네요.', '10'], ['아 진짜? 그러면 고맙지! 완전 땡큐다.', '0'], ['맞아. 밖에 나가서 햇빛을 못 보니 몸이 지치고 마음은 더 힘들고.', '0'], ['그래. 그동안 친구를 못만났으니까 못다했던 이야기도 하고 재밌게 놀고 와야겠어.', '0'], ['밖에 잠깐 나갔다 왔지. 내일 비가 오는 거 맞어? 우산은 항상 있어. 걱정 안 해.', '0'], ['역겨운 냄새가 나고 있어.', '0'], ['나 자다가 정말 깜짝 놀랐어!', '0'], ['온열 꺼줘.', '10'], ['사과 영상을 봤는데 사과라기보다는 자기 변명만 하고 있어서 더 화가 났어.', '0'], ['쓰레기통에서 역겨운 냄새가 나고 있어.', '0'], ['밖에 비가 엄청 와서.', '0'], ['아니. 아까 해피를 산책시키다가 깜짝 놀랐어.', '0'], ['하이 코웨이.', '1'], ['그런 것도 있어? 한 번 추천해주면 고맙지.', '0'], ['그런게 있으면 지금 바로 가입해야지.', '0'], ['하이 코웨이.', '1'], ['자고 일어났더니 집 주변이 다 물바다가 되어있어.', '0'], ['나 혼자 고립돼서 갇힌 것 같애.', '0'], ['하이 코웨이.', '1'], ['하이 코웨이.', '1'], ['어, 괜찮아.', '0'], ['등이땡겨.', '16'], ['아니, 원래는 바닷가 갈려고 했지.', '0'], ['짭새가 몰려드는게 이상한거긴 해. 나도 구경 잘 못했어.', '0'], ['온열 꺼줄래 너무 덥다.', '10'], ['어깨가안좋아.', '15'], ['힘들다.', '13'], ['화장실은 청소를 해도 곰팡이가 없어지질 않아.', '0'], ['멀쩡하던 목줄이 갑자기 끊어져버린거야.', '0'], ['음악을 들어도 기분이 좋아질 것 같진 않아. 하지만 하나 골라줘.', '0'], ['하이 코웨이.', '1'], ['오를 줄 알았는데 떨어지더라고.', '0'], ['힘이들어.', '13'], ['등이땡겨.', '16'], [

In [53]:
print(len(dataset_test))
print(dataset_test)

for data, label in dataset_test:
    print(type(data))
    print(data)
    print(label)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
나 너무 우울해. 매일매일이 힘들어.
0
<class 'str'>
프로젝트를 하느라고 한 달 동안 친구를 못 만났는데 이번 기회에 만나봐야겠어.
0
<class 'str'>
하이 코웨이.
1
<class 'str'>
톡방에 지인이 권유하더라고.
0
<class 'str'>
한 10분에서 15분 정도 구경했는데 별 건 없더라고?
0
<class 'str'>
맞아. 우리 집은 화장실에 창문 하나도 없어. 그러니 습기가 빠져나갈 데가 없어.
0
<class 'str'>
하이 코웨이.
1
<class 'str'>
집에서 쉬고 나니까 마음이 진정되었어.
0
<class 'str'>
나 지금 기분이 너무 안좋은데 어떤 영상을 추천해줄건데?
0
<class 'str'>
어깨가땡겨.
15
<class 'str'>
쓰레기 냄새랑 섞여서 이상하지 않을까?
0
<class 'str'>
무중력모드 해주세요.
7
<class 'str'>
마라톤에서 기록을 갱신했어!
0
<class 'str'>
나 주식 시작했는데 후회하고 있어.
0
<class 'str'>
내 휴가가 엉망이야.
0
<class 'str'>
드디어 새로 산 노트북이 오늘 도착했어.
0
<class 'str'>
나 너무 우울해.
0
<class 'str'>
등이결려.
16
<class 'str'>
무슨 일인지는 잘 모르겠어.
0
<class 'str'>
당연히 동해바다 갈려 그랬지.
0
<class 'str'>
내가 잠이 들면 업어가도 몰라. 그래서 재난 문자랑 피하라는 전화도 못 받고, 그냥 잠만 자고 있었나 봐.
0
<class 'str'>
온열 오프해주세요.
10
<class 'str'>
오랜만에 가족들이랑 동해 바다에 가서 놀다 오기로 했어. 그런데 망했어.
0
<class 'str'>
아니, 요즘 왜 그러는거지 다들? 이해가 안되네.
0
<class 'str'>
내 휴가가 엉망이야.
0
<class 'str'>
알았어. 지금이라도 바로 묶어서 버릴

In [54]:
# ★
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,vocab, max_len,
                 pad, pair):

        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))


    def __len__(self):
        return (len(self.labels))

In [55]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [56]:
# ★
tok = tokenizer.tokenize

data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, vocab, max_len, True, False)
print(data_train[0])
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

(array([   2, 4368,  517, 5846, 5703,   54,    3,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32), array(7, dtype=int32), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=int32), 10)


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [57]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 17, # 클래스 수 조정
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [58]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [59]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [60]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [61]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
train_dataloader

In [62]:
# 머신러닝 학습시작
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

<ipython-input-62-c4b88bee55b1>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/95 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


epoch 1 batch id 1 loss 2.8554954528808594 train acc 0.0
epoch 1 train acc 0.6478383458646616


<ipython-input-62-c4b88bee55b1>:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


...

epoch 4 test acc 1.0


  0%|          | 0/95 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.023664528504014015 train acc 1.0
epoch 5 train acc 0.9990131578947369


  0%|          | 0/32 [00:00<?, ?it/s]

epoch 5 test acc 1.0


In [63]:
# 최초 1회 설정
import firebase_admin
from firebase_admin import credentials
# The Cloud Functions for Firebase SDK to create Cloud Functions and set up triggers.
from firebase_functions import db_fn, https_fn

# The Firebase Admin SDK to access the Firebase Realtime Database.
from firebase_admin import initialize_app, db

cred = credentials.Certificate('/content/drive/MyDrive/coway-chair-test-server-firebase-adminsdk-hhu3y-2c4d064bcf.json')

try:
    app = firebase_admin.initialize_app(cred, { 'databaseURL' : "https://coway-chair-test-server-default-rtdb.firebaseio.com/" })
except ValueError:
    app = firebase_admin.initialize_app    # 그대로 admin 사용

In [64]:
# firebase에 데이터 쓰기
# dir = db.reference()
# dir.update({"Result" : "testtest"})

In [65]:
# firebase에 데이터 읽어오기
dir = db.reference('Result')
print(dir.get())

None


In [66]:
from typing import Any
@db_fn.on_value_created(reference="/Result")
def makeuppercase(event: db_fn.Event[Any]) -> None:
    """Listens for new messages added to /messages/{pushId}/original and
    creates an uppercase version of the message to /messages/{pushId}/uppercase
    """

    # Grab the value that was written to the Realtime Database.
    original = event.data
    if not isinstance(original, str):
        print(f"Not a string: {event.reference}")
        return

    # Use the Admin SDK to set an "uppercase" sibling.
    print(f"Uppercasing {event.params['pushId']}: {original}")
    upper = original.upper()
    parent = db.reference(event.reference).parent
    if parent is None:
        print("Message can't be root node.")
        return
    parent.child("uppercase").set(upper)

In [67]:
from firebase_functions.core import CloudEvent
@db_fn.on_value_written(reference="hello/world")
def onwriteexample(event: db_fn.Event[db_fn.Change]) -> None:
    print("Hello from db write event:", event)


@db_fn.on_value_created(reference="hello/world")
def oncreatedexample(event: db_fn.Event) -> None:
    print("Hello from db create event:", event)


@db_fn.on_value_deleted(reference="hello/world")
def ondeletedexample(event: db_fn.Event) -> None:
    print("Hello from db delete event:", event)


@db_fn.on_value_updated(reference="hello/world")
def onupdatedexample(event: db_fn.Event[db_fn.Change]) -> None:
    print("Hello from db updated event:", event)

In [68]:
#토큰화
# tokenizer = tokenizer.tokenize
# tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    if predict_sentence == '0':
        return

    # firebase에 데이터 쓰기
    dir = db.reference()
    data = [predict_sentence, '0']
    dataset_another = [data]
    print(dataset_another)
    another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)

    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)

        test_eval=[]
        for i in out:
            logits = i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 1:
                test_eval.append("사용자 코웨이 AI 호출")

                dir.child("NLU").update({"Command" : "null"})
                dir.child("NLU").update({"Result" : "COWAY"})

            elif np.argmax(logits) == 2:
                test_eval.append("안마의자 전원 ON 명령")

                dir.child("NLU").update({"Command" : "null"})
                dir.child("NLU").update({"Result" : "POWER_ON"})

            elif np.argmax(logits) == 3:
                test_eval.append("안마의자 전원 OFF 명령")

                dir.child("NLU").update({"Command" : "null"})
                dir.child("NLU").update({"Result" : "POWER_OFF"})

            elif np.argmax(logits) == 4:
                test_eval.append("Good Moring모드 동작 명령")

                dir.child("NLU").update({"Command" : "null"})
                dir.child("NLU").update({"Result" : "GOOD_MORING"})

            elif np.argmax(logits) == 5:
                test_eval.append("Good Day모드 동작 명령")

                dir.child("NLU").update({"Command" : "null"})
                dir.child("NLU").update({"Result" : "GOOD_DAY"})

            elif np.argmax(logits) == 6:
                test_eval.append("Good night모드 동작 명령")

                dir.child("NLU").update({"Command" : "null"})
                dir.child("NLU").update({"Result" : "GOOD_NIGHT"})

            elif np.argmax(logits) == 7:
                test_eval.append("무중력모드 동작 명령")

                dir.child("NLU").update({"Command" : "null"})
                dir.child("NLU").update({"Result" : "ZEROG"})

            elif np.argmax(logits) == 8:
                test_eval.append("스윙모드 동작 명령")

                dir.child("NLU").update({"Command" : "null"})
                dir.child("NLU").update({"Result" : "SWING"})

            elif np.argmax(logits) == 9:
                test_eval.append("온열 기능 ON 명령")

                dir.child("NLU").update({"Command" : "null"})
                dir.child("NLU").update({"Result" : "HEAT_ON"})

            elif np.argmax(logits) == 10:
                test_eval.append("온열 기능 OFF 명령")

                dir.child("NLU").update({"Command" : "null"})
                dir.child("NLU").update({"Result" : "HEAT_OFF"})

            elif np.argmax(logits) == 11:
                test_eval.append("사용자 응답 (동의)")

                dir.child("NLU").update({"Command" : "null"})
                dir.child("NLU").update({"Result" : "USER_AGREE"})

            elif np.argmax(logits) == 12:
                test_eval.append("사용자 응답 (비동의)")

                dir.child("NLU").update({"Command" : "null"})
                dir.child("NLU").update({"Result" : "USER_DISAGREE"})

            elif np.argmax(logits) == 13:
                test_eval.append("사용자 컨디션 (피로)")

                dir.child("NLU").update({"Command" : "null"})
                dir.child("NLU").update({"Result" : "USER_TIRED"})

            elif np.argmax(logits) == 14:
                test_eval.append("사용자 목안마 필요")

                dir.child("NLU").update({"Command" : "null"})
                dir.child("NLU").update({"Result" : "USER_NEED_NECK_MSG"})

            elif np.argmax(logits) == 15:
                test_eval.append("사용자 어깨안마 필요")

                dir.child("NLU").update({"Command" : "null"})
                dir.child("NLU").update({"Result" : "USER_NEED_SHOULDER_MSG"})

            elif np.argmax(logits) == 16:
                test_eval.append("사용자 등안마 필요")

                dir.child("NLU").update({"Command" : "null"})
                dir.child("NLU").update({"Result" : "USER_NEED_BACK_MSG"})

            else:
                test_eval.append("일상 대화")
        print(test_eval[0])

In [69]:
#질문 무한반복하기! 0 입력시 종료
end = 1
data = ""
data_old = ""
flag = 1
while end == 1 :
    time.sleep(1)
    # firebase에 데이터 읽어오기
    dir = db.reference().child("NLU").child("Command")
    data = dir.get()

    if(data_old != data):
        data_old = data

        if data == '0.' and flag == 0:          # 0을 입력하면 종료
            print("종료합니다.")
            end = 0
        elif data == "null" or data == ".":     # 데이터를 보냈거나 음성이 제대로 감지되지 않았을 때는 무시
            pass
        else:
            try:
                flag = 0
                predict(data)
            except TypeError:
                pass
    # sentence = input("하고싶은 말을 입력해주세요 : ")
    # if sentence == '0' :
    #     print("종료합니다.")
    #     end = 0
    # predict(sentence)
    # print("\n")

[['피곤해.', '0']]
사용자 컨디션 (피로)
[['형님재밌는거보여드릴까요.', '0']]
일상 대화
[['위에거하이코웨이.', '0']]
사용자 코웨이 AI 호출
[['이거디자인할사람만구하면되잖아피곤해.', '0']]
일상 대화
[['피곤해.', '0']]
사용자 컨디션 (피로)
[['목이걸려.', '0']]
사용자 목안마 필요
[['네네.', '0']]
사용자 응답 (비동의)


KeyboardInterrupt: 